# **Import & Load**

In [ ]:
import pandas as pd

file_id = "1T_GLCDbLTzi3rTVut7RHT8S0rsNXuGyi"
url = f"https://drive.google.com/uc?id={file_id}"

df = pd.read_csv(url)

print("Shape:", df.shape)
display(df.head())
print("\nKolom:", df.columns.tolist())

Shape: (19931, 25)


,myanimelist_id,title,description,image,Type,Episodes,Status,Premiered,Released_Season,Released_Year,...,Demographic,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,characters,source_url
0,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,TV,26,Finished Airing,Spring 1998,Spring,1998.0,...,NaN,24 min. per ep.,R - 17+ (violence & profanity),8.75,#48,#42,"2,008,019","87,916","[{""id"": 3, ""name"": ""Black, Jet"", ""url"": ""https...",https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...",https://cdn.myanimelist.net/images/anime/1439/...,Movie,1.0,Finished Airing,NaN,NaN,NaN,...,NaN,1 hr. 55 min.,R - 17+ (violence & profanity),8.38,#232,#649,"403,604","1,748","[{""id"": 3, ""name"": ""Black, Jet"", ""url"": ""https...",https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0...",https://cdn.myanimelist.net/images/anime/1130/...,TV,26,Finished Airing,Spring 1998,Spring,1998.0,...,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,8.22,#385,#265,"815,140","17,193","[{""id"": 713, ""name"": ""Stryfe, Meryl"", ""url"": ""...",https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,"Though hidden away from the general public, Wi...",https://cdn.myanimelist.net/images/anime/10/19...,TV,26.0,Finished Airing,Summer 2002,Summer,2002.0,...,NaN,25 min. per ep.,PG-13 - Teens 13 or older,7.23,#3344,#1979,"125,868",686,"[{""id"": 300, ""name"": ""Amon"", ""url"": ""https://m...",https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,It is the dark century and the people are suff...,https://cdn.myanimelist.net/images/anime/7/215...,TV,52,Finished Airing,Fall 2004,Fall,2004.0,...,Shounen,23 min. per ep.,PG - Children,6.92,#4887,#5765,"16,456",18,"[{""id"": 9054, ""name"": ""Beet"", ""url"": ""https://...",https://myanimelist.net/anime/8/Bouken_Ou_Beet



Kolom: ['myanimelist_id', 'title', 'description', 'image', 'Type', 'Episodes', 'Status', 'Premiered', 'Released_Season', 'Released_Year', 'Source', 'Genres', 'Themes', 'Studios', 'Producers', 'Demographic', 'Duration', 'Rating', 'Score', 'Ranked', 'Popularity', 'Members', 'Favorites', 'characters', 'source_url']


# **Kolom genre & target main_genre**

In [ ]:
if "Genres" not in df.columns:
    raise ValueError("Kolom 'Genres' tidak ditemukan. Cek nama kolom di df.columns!")

# ambil genre utama (sebelum koma pertama)
df["main_genre"] = df["Genres"].astype(str).str.split(",").str[0].str.strip()

# bersihin nilai kosong
df = df[df["main_genre"].notna()]
df = df[df["main_genre"] != ""]
df = df[df["main_genre"] != "nan"]

print("Contoh main_genre:")
display(df[["Genres","main_genre"]].head(10))
print("\nJumlah kelas (unique):", df["main_genre"].nunique())
display(df["main_genre"].value_counts().head(15))

Contoh main_genre:


,Genres,main_genre
0,"Action, Award Winning, Sci-Fi",Action
1,"Action, Sci-Fi",Action
2,"Action, Adventure, Sci-Fi",Action
3,"Action, Drama, Mystery, Supernatural",Action
4,"Action, Adventure, Fantasy",Action
5,Sports,Sports
6,"Comedy, Drama, Romance",Comedy
7,Sports,Sports
8,"Action, Drama",Action
9,"Drama, Mystery, Suspense",Drama



Jumlah kelas (unique): 21


,count
main_genre,
Comedy,4961
Action,4291
Adventure,2201
Drama,1357
Hentai,1284
Fantasy,1049
Slice of Life,665
Sci-Fi,650
Sports,371


# **Filter TOP-K genre**

In [ ]:
TOP_K = 8  # boleh 5/8/10, rekomendasi 8 biar masih kuat
top_genres = df["main_genre"].value_counts().head(TOP_K).index

df = df[df["main_genre"].isin(top_genres)].copy()

print("Shape setelah filter TOP_K:", df.shape)
print("Distribusi kelas:")
display(df["main_genre"].value_counts())

Shape setelah filter TOP_K: (16458, 26)
Distribusi kelas:


,count
main_genre,
Comedy,4961
Action,4291
Adventure,2201
Drama,1357
Hentai,1284
Fantasy,1049
Slice of Life,665
Sci-Fi,650


# **Pilih fitur metadata**

In [ ]:
# kolom yang umumnya "tidak dipakai"
drop_cols_candidates = [
    "anime_id", "mal_id", "uid", "title", "name", "synopsis", "background",
    "genre",  # genre asli kita ubah jadi main_genre (target)
]

drop_cols = [c for c in drop_cols_candidates if c in df.columns]

# target
target_col = "main_genre"

# pisahkan X dan y
X = df.drop(columns=drop_cols + [target_col], errors="ignore")
y = df[target_col].copy()

print("Drop cols:", drop_cols)
print("X shape:", X.shape, "y shape:", y.shape)
display(X.head())

Drop cols: ['title']
X shape: (16458, 24) y shape: (16458,)


,myanimelist_id,description,image,Type,Episodes,Status,Premiered,Released_Season,Released_Year,Source,...,Demographic,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,characters,source_url
0,1,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,TV,26,Finished Airing,Spring 1998,Spring,1998.0,Original,...,NaN,24 min. per ep.,R - 17+ (violence & profanity),8.75,#48,#42,"2,008,019","87,916","[{""id"": 3, ""name"": ""Black, Jet"", ""url"": ""https...",https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,"Another day, another bounty—such is the life o...",https://cdn.myanimelist.net/images/anime/1439/...,Movie,1.0,Finished Airing,NaN,NaN,NaN,Original,...,NaN,1 hr. 55 min.,R - 17+ (violence & profanity),8.38,#232,#649,"403,604","1,748","[{""id"": 3, ""name"": ""Black, Jet"", ""url"": ""https...",https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,"Vash the Stampede is the man with a $$60,000,0...",https://cdn.myanimelist.net/images/anime/1130/...,TV,26,Finished Airing,Spring 1998,Spring,1998.0,Manga,...,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,8.22,#385,#265,"815,140","17,193","[{""id"": 713, ""name"": ""Stryfe, Meryl"", ""url"": ""...",https://myanimelist.net/anime/6/Trigun
3,7,"Though hidden away from the general public, Wi...",https://cdn.myanimelist.net/images/anime/10/19...,TV,26.0,Finished Airing,Summer 2002,Summer,2002.0,Original,...,NaN,25 min. per ep.,PG-13 - Teens 13 or older,7.23,#3344,#1979,"125,868",686,"[{""id"": 300, ""name"": ""Amon"", ""url"": ""https://m...",https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,It is the dark century and the people are suff...,https://cdn.myanimelist.net/images/anime/7/215...,TV,52,Finished Airing,Fall 2004,Fall,2004.0,Manga,...,Shounen,23 min. per ep.,PG - Children,6.92,#4887,#5765,"16,456",18,"[{""id"": 9054, ""name"": ""Beet"", ""url"": ""https://...",https://myanimelist.net/anime/8/Bouken_Ou_Beet


# **Deteksi numerik/kategorikal + preprocessing pipeline**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# deteksi tipe kolom
num_cols = X.select_dtypes(include=["int64","float64","int32","float32"]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

print("Numerik:", num_cols)
print("Kategorikal:", cat_cols)

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop"
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)

Numerik: ['myanimelist_id', 'Released_Year', 'Score']
Kategorikal: ['description', 'image', 'Type', 'Episodes', 'Status', 'Premiered', 'Released_Season', 'Source', 'Genres', 'Themes', 'Studios', 'Producers', 'Demographic', 'Duration', 'Rating', 'Ranked', 'Popularity', 'Members', 'Favorites', 'characters', 'source_url']
Train: (13166, 24) Test: (3292, 24)


# **Fit preprocess & simpan (buat dipakai 3 model + Streamlit)**

In [ ]:
import joblib

preprocess.fit(X_train)

X_train_t = preprocess.transform(X_train)
X_test_t  = preprocess.transform(X_test)

print("Setelah transform:")
print("X_train_t:", X_train_t.shape)
print("X_test_t :", X_test_t.shape)

joblib.dump(preprocess, "preprocess.pkl")
joblib.dump(list(top_genres), "top_genres.pkl")
joblib.dump((num_cols, cat_cols), "feature_cols.pkl")

print("Saved: preprocess.pkl, top_genres.pkl, feature_cols.pkl")

Setelah transform:
X_train_t: (13166, 86876)
X_test_t : (3292, 86876)
Saved: preprocess.pkl, top_genres.pkl, feature_cols.pkl
